In [2]:
import torch

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device, dtype=torch.float32)
    freqs = torch.outer(t, freqs)
    print(freqs.shape)
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    print(freqs_cis.shape)
    return freqs_cis

precompute_freqs_cis(16, 10)

torch.Size([10, 8])
torch.Size([10, 8])


tensor([[ 1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j,  1.0000+0.0000e+00j],
        [ 0.5403+8.4147e-01j,  0.9504+3.1098e-01j,  0.9950+9.9833e-02j,
          0.9995+3.1618e-02j,  0.9999+9.9998e-03j,  1.0000+3.1623e-03j,
          1.0000+1.0000e-03j,  1.0000+3.1623e-04j],
        [-0.4161+9.0930e-01j,  0.8066+5.9113e-01j,  0.9801+1.9867e-01j,
          0.9980+6.3203e-02j,  0.9998+1.9999e-02j,  1.0000+6.3245e-03j,
          1.0000+2.0000e-03j,  1.0000+6.3246e-04j],
        [-0.9900+1.4112e-01j,  0.5828+8.1265e-01j,  0.9553+2.9552e-01j,
          0.9955+9.4726e-02j,  0.9995+2.9995e-02j,  1.0000+9.4867e-03j,
          1.0000+3.0000e-03j,  1.0000+9.4868e-04j],
        [-0.6536-7.5680e-01j,  0.3011+9.5358e-01j,  0.9211+3.8942e-01j,
          0.9920+1.2615e-01j,  0.9992+3.9989e-02j,  0.9999+1.2649e-02j,
          1.0000+4.0000e-03j,  1.0000+1.2649e-03j],
        [ 0.2837-9.5

In [3]:
import torch
a = torch.randn(2, 3).to(torch.float16)
s = a.sum()
a


tensor([[ 0.1812, -1.3740,  0.1895],
        [-0.7515, -0.5161,  0.5698]], dtype=torch.float16)

In [3]:
import torch
model_weight = torch.load("/home/gexingt/tgx/models/Llama-2-7b-hf/pytorch_model-00001-of-00002.bin")
model_weight['model.embed_tokens.weight'].shape


/tmp/ipykernel_22546/175741966.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weight = torch.load("/home/gexingt/tgx/models/Llama-2-7b-hf/pytorch_model-00001-of-0

torch.Size([32000, 4096])

In [ ]:
torch.transpose

In [3]:
model_weight['model.layers.0.self_attn.rotary_emb.inv_freq'].shape

torch.Size([64])

In [6]:
model_weight['model.layers.1.self_attn.rotary_emb.inv_freq'].dtype

torch.float32

In [5]:
for i in range(0, 64):
    freq = 1.0 / (10000 ** (i / 64))
    print(freq)


1.0
0.8659643233600653
0.7498942093324559
0.6493816315762113
0.5623413251903491
0.48696752516586306
0.4216965034285823
0.36517412725483767
0.31622776601683794
0.27384196342643613
0.23713737056616555
0.2053525026457146
0.17782794100389226
0.1539926526059492
0.1333521432163324
0.1154781984689458
0.1
0.08659643233600654
0.07498942093324558
0.06493816315762113
0.056234132519034905
0.04869675251658631
0.042169650342858224
0.03651741272548377
0.03162277660168379
0.02738419634264361
0.02371373705661655
0.020535250264571463
0.01778279410038923
0.01539926526059492
0.01333521432163324
0.011547819846894583
0.01
0.008659643233600653
0.007498942093324559
0.006493816315762114
0.005623413251903491
0.004869675251658632
0.004216965034285823
0.003651741272548377
0.003162277660168379
0.0027384196342643613
0.002371373705661655
0.002053525026457146
0.001778279410038923
0.0015399265260594918
0.001333521432163324
0.0011547819846894581
0.001
0.0008659643233600654
0.0007498942093324557
0.0006493816315762112
0.

In [4]:
import torch
model_weight = torch.load("/home/gexingt/tgx/models/Llama-2-7b-hf/pytorch_model-00001-of-00002.bin")
inv_freq = model_weight['model.layers.0.self_attn.rotary_emb.inv_freq']

def get_cos_sin(x, position_ids, inv_freq):
    inv_freq_expanded = inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
    position_ids_expanded = position_ids[:, None, :].float()
    freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
    emb = torch.cat((freqs, freqs), dim=-1)
    cos = emb.cos()
    sin = emb.sin()
    return cos, sin

def get_cos_sin_half(x, position_ids, inv_freq):
    inv_freq_expanded = inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
    position_ids_expanded = position_ids[:, None, :].float()
    freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
    emb = freqs
    cos = emb.cos()
    sin = emb.sin()
    return cos, sin

def rotate_half(x):
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, cos, sin, position_ids=None, unsqueeze_dim=1):
    """Applies Rotary Position Embedding to the query and key tensors.

    Args:
        q (`torch.Tensor`): The query tensor.
        k (`torch.Tensor`): The key tensor.
        cos (`torch.Tensor`): The cosine part of the rotary embedding.
        sin (`torch.Tensor`): The sine part of the rotary embedding.
        position_ids (`torch.Tensor`, *optional*):
            Deprecated and unused.
        unsqueeze_dim (`int`, *optional*, defaults to 1):
            The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
            sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
            that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
            k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
            cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
            the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
    Returns:
        `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
    """
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    return q_embed

q = torch.randn(1, 32, 1024, 128)
position = torch.arange(1024).unsqueeze(0)
cos, sin = get_cos_sin(q, position, inv_freq)
print(cos.shape, sin.shape)
cos_half, sin_half = get_cos_sin_half(q, position, inv_freq)
print(cos_half.shape, sin_half.shape)
q_embed = apply_rotary_pos_emb(q, cos, sin, position, unsqueeze_dim=1)
print(q_embed.shape)





/tmp/ipykernel_27170/1918952304.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weight = torch.load("/home/gexingt/tgx/models/Llama-2-7b-hf/pytorch_model-00001-of-

torch.Size([1, 1024, 128]) torch.Size([1, 1024, 128])
torch.Size([1, 1024, 64]) torch.Size([1, 1024, 64])
torch.Size([1, 32, 1024, 128])


In [2]:
import torch
batch_size = 1
seq_len = 16
dim = 32
intern_dim = 64
x = torch.randn(batch_size, seq_len, dim, dtype=torch.float32)
gate_proj = torch.randn(intern_dim, dim, dtype=torch.float32)
up_proj = torch.randn(intern_dim, dim, dtype=torch.float32)
down_proj = torch.randn(dim, intern_dim, dtype=torch.float32)

def pytorch_ffn(x, gate_proj, up_proj, down_proj):
    #silu ffn
    up_proj_out = x @ up_proj.t()
    gate_proj_out = x @ gate_proj.t()
    gate_proj_out = gate_proj_out * torch.sigmoid(gate_proj_out)
    gate_proj_out = up_proj_out * gate_proj_out
    down_proj_out = gate_proj_out @ down_proj.t()
    return down_proj_out

result_py = pytorch_ffn(x, gate_proj, up_proj, down_proj)

In [1]:
length = 4096
bsz = 1
dims = 4096

tensor_num = 9


#9 * 4096 * 4096 * 4 / 1024 / 1024 / 1024

0.5625

In [4]:
# prefill
length = 1024
bsz = 1
dims = 4096
tensor_num = 7
kvsize = 4096 * 2 * 4 * 4096
attnsize = length * length * 32 * bsz * 4
all_size = (tensor_num * length * dims * 4 + kvsize + attnsize) / 1024/ 1024
all_size

368.0

In [6]:
#decode
attnsize = 4096 * 32 * bsz * 4
all_size = (2 * 4096 * dims * 4 + kvsize + 5 * dims * bsz * 4 + 4096 * 32 * 4) / 1024 / 1024
all_size

256.578125